# **CleanWaterAI** – Water Potability Predictor

This notebook trains a neural network to predict whether water is safe to drink based on 9 chemical features.  
Aligned with **UN Sustainable Development Goal 6**: *Clean Water and Sanitation*.

**1.Importing Libraries**

In [26]:
# Step 1: Import Libraries
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**2.Load and preprocess the data**

In [27]:
# Step 2: Load and preprocess the data
df = pd.read_csv("/content/water_potability.csv")
df.fillna(df.mean(), inplace=True)  # Fill missing values with column averages

X = df.drop("Potability", axis=1)
y = df["Potability"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

**3.Define the Neural Network**

In [28]:
# Step 3: Define the Neural Network
class WaterNet(nn.Module):
    def __init__(self, input_dim):
        super(WaterNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

 **4.Prepare data for PyTorch**

In [29]:
# Step 4: Prepare data for PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

5. **Initialize model, loss function and optimizer**

In [30]:
# Step 5: Initialize model, loss function and optimizer
model = WaterNet(input_dim=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

**6. Train the model**

In [31]:
# Step 6: Train the model
for epoch in range(20):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/20 - Loss: {total_loss:.4f}")

Epoch 1/20 - Loss: 54.9479
Epoch 2/20 - Loss: 54.1319
Epoch 3/20 - Loss: 53.3228
Epoch 4/20 - Loss: 51.7247
Epoch 5/20 - Loss: 51.0190
Epoch 6/20 - Loss: 50.9817
Epoch 7/20 - Loss: 49.9552
Epoch 8/20 - Loss: 49.4736
Epoch 9/20 - Loss: 49.4662
Epoch 10/20 - Loss: 49.6186
Epoch 11/20 - Loss: 49.0592
Epoch 12/20 - Loss: 48.5848
Epoch 13/20 - Loss: 48.9299
Epoch 14/20 - Loss: 48.5776
Epoch 15/20 - Loss: 48.5694
Epoch 16/20 - Loss: 48.2958
Epoch 17/20 - Loss: 48.5343
Epoch 18/20 - Loss: 48.6881
Epoch 19/20 - Loss: 48.1184
Epoch 20/20 - Loss: 47.8773


**7. Evaluate the model**

In [32]:
# Step 7: Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = (y_pred > 0.5).float()
    acc = (y_pred_class == y_test_tensor).float().mean()
    print(f"\n✅ Test Accuracy: {acc.item() * 100:.2f}%")





✅ Test Accuracy: 68.45%


**8. Saving the model**

In [33]:
#Step 8: Saving the model
torch.save(model.state_dict(), "water_model.pt")
print("\nModel saved as 'water_model.pt'")


Model saved as 'water_model.pt'


**9. Saving the model**

In [34]:
# Correct way to save model weights
torch.save(model.state_dict(), "water_model.pt")
from google.colab import files
files.download("water_model.pt")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### ✅ Summary

- Trained a neural network using PyTorch
- Achieved ~68% test accuracy
- Saved the model for use in prediction

Ready to real-world testing !
